# In-Class Coding Lab: Databases

The goals of this lab are to help you to understand:

- Using the SqlLite database engine to store relational data
- Create a database and tables for your application
- Saving, retrieving, updating, and deleting data in the database


A database is a file that is organized for storing data. Most databases are organized like a dictionary in the sense that they map from keys to values. Databases are build using columns and rows, just like and excel file. The other great thing about databases is they persist. Just like files your data will be available again if you have to restart your program.

## Our Favs

For our program we have decided to build a list of our favorite music tracks. Our goal is to create a database that contains a list of our favorite songs and artists. To do this we will use a database to store all the data we submit so we can look it up later.

First, we will need to model our data. Data modeling gives us the opportunity to think through our data and develop a schema for our database. This approach takes much less time then recreating a new database everytime you realize you need to add a new column to your table. It also allows you to think through complex relationships and how to build your database to best represent them. Data modeling can be a sophisticated problem, for our use case we will keep it simple. If you want a quick lesson on data modeling [check this out](http://www.agiledata.org/essays/dataModeling101.html)


First step in modeling data is breaking your data entity into separate properties. Databases require you to define the type of data that it is storing in each column. So for our favs, we would need to have some sort of identifier, the title of the song, and the artist. Each property needs to be assigned a type. For example

- favid: Interger (Identifier, This is called a primary key)
- title: string
- artist: string

Now that we have a simple model for our data, we need to create our database.

In [4]:
import sqlite3

conn = sqlite3.connect('myfavs_test.sqlite3')
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS Favs''')

# Below we create the table, the AUTOINCREMENT keywork tells the database to track our identifier
cur.execute('''CREATE TABLE Favs (favid INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT, artist Text)''')

conn.close()

To view the database that we just created download and install the [sqllitebrowser](http://sqlitebrowser.org/). If you look in the directory that this file is contained in, you should see a new file called myfavs_text.sqllite3. When you open the database you will see a new table called Favs with the columns that we specified.

Now that we have a database lets add a fav to it. To interact with a database we use SQL or Structured Query Language. Since there are so many different database vendors, SQL was standardized so we could communicate with systems from multiple vendors.

There is a great tutorial on SQL at [W3Schools](https://www.w3schools.com/SQL/deFault.asp). We will be covering the basics in
this lab, but I encourage you to check out the tutorial in your free time.

### Inserting Data

First we need to add a fav to the database. To do this we use the INSERT statement.

In [7]:
import sqlite3

# Connect to our database
conn = sqlite3.connect('myfavs_test.sqlite3')
# get a cursor
cur = conn.cursor()
# Execute a command
cur.execute('''INSERT INTO Favs (title, artist) VALUES (?, ?)''', ('Smells Like Teen Spirit', 'Nirvana'))
# Commits changes
conn.commit()
conn.close()

Open the database to see the row inserted into the Favs table. So what just happened? Lets walk through each step.

1. Import the sqllite3 library
2. Connect to the database, place the connection in the conn variable
3. Get a cursor from the database, a cursor gets used to execute commands to the database
4. Execute the insert command, we place our command in ''', this allows us to use ' and " quotes in our sql commands without having to worry about escaping that character.
5. when you update data in the database you need to commit it. this allows you to undo the changes before you actually do any writing, in the database world this is called a transaction.
6. Close the connection

You are probably also wondering why we use the ?'s in our SQL statement? This protects your program against an attack call sql injection, this is an attack where someone can injet there own sql code into your command to execute there own commands on your database, such as deleting all of your data.

Now you try to insert one of your favs

In [8]:
# Write your code here.



### Retrieving Data

Now that we have some data in our database, lets try to read it back out so we can use it in our program. 

To do this we use a "select" statement. Everything else is the same. When you read data from a database the data is returned as a list of rows.

In [9]:
import sqlite3
conn = sqlite3.connect('myfavs_test.sqlite3')
cur = conn.cursor()
# Execute a select command
rows = cur.execute('''SELECT * FROM Favs''')

# Iterate over the results
for row in rows:
    print(row)


conn.close()

(1, 'Smells Like Teen Spirit', 'Nirvana')


So far so good! So what if we want to only retrieve a filtered amount of data? we use the WHERE keyword and a condition.

In [11]:
import sqlite3
conn = sqlite3.connect('myfavs_test.sqlite3')
cur = conn.cursor()
# Execute a select command
rows = cur.execute('''SELECT * FROM Favs WHERE artist=?''', ('Nirvana',))

# Iterate over the results
for row in rows:
    print(row)


conn.close()

(1, 'Smells Like Teen Spirit', 'Nirvana')


Now you try to get all of your favs by your artist.

In [ ]:
# Write your code here



So what if we just want 1 result? We would use this if we wanted to get a record that we needed to update (More on this in a min). To do this we use the LIMIT keyword in our sql statement and the fetchone() method of our cursor object. USUALLY we use the primary key to look up a single record.

fetchone() will return None if a record is not found, in most programs we would want to check for this and let the user know that your program couldn't find that record. 

In the example below, change the 1 to 100 in our sql statement, and see the result.

In [13]:
import sqlite3
conn = sqlite3.connect('myfavs_test.sqlite3')
cur = conn.cursor()
# Execute a select command
row = cur.execute('''SELECT * FROM Favs WHERE favid=?''', (1,)).fetchone()

# row will be a single row or None. In a real program we would want to check this
if row is not None:
    print(row)
else:
    print("Row does not exist")


conn.close()

Row does not exist


Now you try, using the results from finding all of the data, fetch your record from the database.

In [ ]:
# Write your code here



### Updating data

A database wouldn't really be useful if you couldn't change the data. So lets do that. The steps required for changing a record in the database are as follows:

- Retrieve the record from the database
- Update values in the data
- Save the data back to the database

Lets start by getting the record we need to change, we use the same code as above, Once you have the data, change the data in the record. and update the record in the database. To do this we use the UPDATE and SET keywords in our sql statement. we need to use the WHERE and provide a condition for which record to update. In our examples we use the primary key 'favid' because this guarantess that we are updating only the record we want to update.

In [26]:
import sqlite3
conn = sqlite3.connect('myfavs_test.sqlite3')

cur = conn.cursor()

# Get the record from the database, When performing and update you should always do this first
row = cur.execute('''SELECT * FROM Favs WHERE favid=?''', (1,)).fetchone()

if row is None:
    print("Record does not exist")

# row is a tuple, tuples are immutable to we need to covert it to a list to change it.
row = list(row)

print("original values: ")
print(row)

# Change the data in the list using the index
# This will change the song name
row[1] = 'Lithium'

# Print the data to see that it changed
print("New values: ")
print(row)

# Now update the record in the database using the UPDATE keyword
cur.execute('''UPDATE Favs SET title=? WHERE favid=?''', (row[1], row[0]))

conn.commit()

conn.close()


original values: 
[1, 'Smells like teen spirit', 'Nirvana']
New values: 
[1, 'Lithium', 'Nirvana']


Now if you open the database in the viewer again, you will see that the data was updated.

Now its your turn!

Write a program that updates more than one record. It's almost the same as above but we will need to add a couple of loops to handle updating many records.

Before you get started, First insert the following songs into your library by running the code below

In [27]:
import sqlite3
conn = sqlite3.connect('myfavs_test.sqlite3')

cur = conn.cursor()

songs = ['Roar', 'Last Friday Night', 'Firework']
artist = 'Matthew Perry'
for song in songs:
    cur.execute('''INSERT INTO Favs (title, artist) VALUES (?, ?)''', (song, artist))
    
conn.commit()
conn.close()

Now if your a big Katie Perry fan, you will notice that we really screwed this up. Write the program to fix this below.

Here are the steps you will need to follow.
- Get all the records from the database
- Loop through each row executing an update for each record.
- Once all of the updates have been execute
- Commit all of the changes to the database
- Close the connection

In [ ]:
# Write your code here!



### Deleting Data

Ok, so we decided that we are no longer big Katie Perry fans, and we don't want her on our list anymore. How do we fix that. 

By useing the DELETE statement, you may need to alter the code below to match how you changed her name in the database

In [31]:
import sqlite3
conn = sqlite3.connect('myfavs_test.sqlite3')
cur = conn.cursor()

# We execute the delete statement below
cur.execute('''DELETE FROM Favs WHERE artist=?''', ('Mathew Perry', ))
    
conn.commit()
conn.close()

Now check your database again and you see that the records for Katie are now gone.

Now you try! 

You decide that Nirvana isn't for you either, write the code below to also remove them from the database.

In [30]:
# Write your code here



### Good job!

Now this lab just illustrates very basic database operations, but hopefully you can see the the power that can be weilded by using databases. The best part is the data persists across usages of your application. Many programmers will use functions encapsulate each functionality.